# 📈 Order Status Metrics Dashboard

**Purpose**: Analyze order status distribution and completion rates for progress tracking.

## Key Metrics
- Order status distribution across all 8 status types
- Quantity completion rates
- Approval/Rejection/Objection rates
- Pipeline visualization

---

## 1. Setup & Data Loading

In [ ]:
# Install required packages
!pip install pandas openpyxl plotly seaborn matplotlib -q

import pandas as pd
import numpy as np
from datetime import datetime
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import seaborn as sns
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)
print("✅ Libraries loaded successfully!")

In [ ]:
# File path - update this to match your file location
filename = r"C:\Users\bmalaraju\Documents\WP-OP Agent\JIRA-Agent\11.25.WP Orders_25-11-2025_v01.xlsx"
print(f"📁 Using file: {filename}")

In [ ]:
# Load data
df = pd.read_excel(filename, engine='openpyxl')
print(f"📊 Dataset loaded: {len(df):,} rows, {len(df.columns)} columns")
print(f"📅 Analysis Date: {datetime.now().strftime('%Y-%m-%d %H:%M')}")

## 2. Data Preparation

In [ ]:
# Column mapping
COLUMN_MAP = {
    'status': 'WP Order Status',
    'product': 'Product',
    'customer': 'Customer',
    'order_id': 'WP Order ID',
    'quantity': 'WP Quantity',
    'completed_qty': 'WP Completed Qty',
    'final_qty': 'WP Final Quantity',
    'approved_qty': 'Total Approved Quantity',
    'project_name': 'Project Name',
    'domain': 'Domain',
    'rejection_reason': 'Approved/Rejected Reason'
}

# Validate columns
missing = [v for v in COLUMN_MAP.values() if v not in df.columns]
if missing:
    print(f"⚠️ Missing columns: {missing}")
else:
    print("✅ All required columns found!")

In [ ]:
# Prepare analysis dataframe
analysis_df = df.copy()

# Normalize status values
status_col = COLUMN_MAP['status']
analysis_df[status_col] = analysis_df[status_col].fillna('Unknown').str.strip()

# Parse quantities with fallbacks
for qty_col in ['quantity', 'completed_qty', 'final_qty', 'approved_qty']:
    col_name = COLUMN_MAP.get(qty_col)
    if col_name and col_name in analysis_df.columns:
        analysis_df[qty_col] = pd.to_numeric(analysis_df[col_name], errors='coerce').fillna(0).astype(int)
    else:
        analysis_df[qty_col] = 0

# Define status categories
STATUS_ORDER = [
    'Pending Acknowledgement',
    'Acknowledge',
    'Waiting for order submission',
    'Pending Approval',
    'Approved',
    'Objected',
    'Rejected',
    'Cancelled'
]

TERMINAL_STATUSES = ['Approved', 'Rejected', 'Cancelled']
IN_PROGRESS_STATUSES = ['Pending Acknowledgement', 'Acknowledge', 'Waiting for order submission', 'Pending Approval', 'Objected']

# Categorize
analysis_df['is_terminal'] = analysis_df[status_col].isin(TERMINAL_STATUSES)
analysis_df['is_approved'] = analysis_df[status_col] == 'Approved'
analysis_df['is_rejected'] = analysis_df[status_col] == 'Rejected'
analysis_df['is_cancelled'] = analysis_df[status_col] == 'Cancelled'
analysis_df['is_objected'] = analysis_df[status_col] == 'Objected'

print(f"✅ Data prepared! Status values found:")
print(analysis_df[status_col].value_counts())

## 3. Overall Status Summary

In [ ]:
# Calculate overall metrics
total_orders = len(analysis_df)
approved = analysis_df['is_approved'].sum()
rejected = analysis_df['is_rejected'].sum()
cancelled = analysis_df['is_cancelled'].sum()
objected = analysis_df['is_objected'].sum()
terminal = analysis_df['is_terminal'].sum()
in_progress = total_orders - terminal

# Quantity metrics
total_qty = analysis_df['quantity'].sum()
completed_qty = analysis_df['completed_qty'].sum()
completion_rate = (completed_qty / total_qty * 100) if total_qty > 0 else 0

# Non-cancelled submission base for approval rate
non_cancelled = total_orders - cancelled
approval_rate = (approved / non_cancelled * 100) if non_cancelled > 0 else 0

# Submitted orders base: anything that got past acknowledgement
submitted_base = approved + rejected + objected
rejection_rate = (rejected / submitted_base * 100) if submitted_base > 0 else 0
objection_rate = (objected / submitted_base * 100) if submitted_base > 0 else 0

# Display summary
print("="*60)
print("📊 ORDER STATUS SUMMARY")
print("="*60)
print(f"\n📋 Total Orders:        {total_orders:,}")
print(f"\n🔄 STATUS BREAKDOWN:")
print(f"   ✅ Approved:         {approved:,} ({approved/total_orders*100:.1f}%)")
print(f"   ❌ Rejected:         {rejected:,} ({rejected/total_orders*100:.1f}%)")
print(f"   🚫 Cancelled:        {cancelled:,} ({cancelled/total_orders*100:.1f}%)")
print(f"   ⚠️ Objected:         {objected:,} ({objected/total_orders*100:.1f}%)")
print(f"   🔄 In Progress:      {in_progress:,} ({in_progress/total_orders*100:.1f}%)")
print(f"\n📈 KEY RATES:")
print(f"   Approval Rate:       {approval_rate:.1f}%")
print(f"   Rejection Rate:      {rejection_rate:.1f}%")
print(f"   Objection Rate:      {objection_rate:.1f}%")
print(f"\n📦 QUANTITY METRICS:")
print(f"   Total Quantity:      {total_qty:,}")
print(f"   Completed Quantity:  {completed_qty:,}")
print(f"   Completion Rate:     {completion_rate:.1f}%")
print("="*60)

In [ ]:
# Status distribution donut chart
status_counts = analysis_df[status_col].value_counts()

# Color mapping for statuses
color_map = {
    'Approved': '#4ECDC4',
    'Rejected': '#FF6B6B',
    'Cancelled': '#95A5A6',
    'Objected': '#FFE66D',
    'Pending Acknowledgement': '#3498DB',
    'Acknowledge': '#2ECC71',
    'Waiting for order submission': '#9B59B6',
    'Pending Approval': '#E67E22',
    'Unknown': '#BDC3C7'
}

colors = [color_map.get(s, '#BDC3C7') for s in status_counts.index]

fig = go.Figure(data=[go.Pie(
    labels=status_counts.index,
    values=status_counts.values,
    hole=0.4,
    marker_colors=colors,
    textinfo='percent+label',
    textposition='outside',
    hovertemplate='%{label}<br>Count: %{value}<br>Percentage: %{percent}<extra></extra>'
)])

fig.update_layout(
    title={'text': 'Order Status Distribution', 'x': 0.5, 'font': {'size': 20}},
    annotations=[{
        'text': f'{total_orders:,}<br>Orders',
        'x': 0.5, 'y': 0.5,
        'font_size': 16,
        'showarrow': False
    }],
    height=600,
    showlegend=True
)

fig.show()

In [ ]:
# KPI Gauges
fig = make_subplots(
    rows=1, cols=3,
    specs=[[{'type': 'indicator'}, {'type': 'indicator'}, {'type': 'indicator'}]],
    subplot_titles=['Approval Rate', 'Completion Rate', 'Terminal %']
)

# Approval Rate
fig.add_trace(go.Indicator(
    mode="gauge+number",
    value=approval_rate,
    gauge={
        'axis': {'range': [0, 100]},
        'bar': {'color': '#4ECDC4'},
        'steps': [
            {'range': [0, 50], 'color': '#FFCDD2'},
            {'range': [50, 75], 'color': '#FFF9C4'},
            {'range': [75, 100], 'color': '#C8E6C9'}
        ]
    },
    number={'suffix': '%'}
), row=1, col=1)

# Completion Rate
fig.add_trace(go.Indicator(
    mode="gauge+number",
    value=completion_rate,
    gauge={
        'axis': {'range': [0, 100]},
        'bar': {'color': '#2ECC71'},
        'steps': [
            {'range': [0, 50], 'color': '#FFCDD2'},
            {'range': [50, 75], 'color': '#FFF9C4'},
            {'range': [75, 100], 'color': '#C8E6C9'}
        ]
    },
    number={'suffix': '%'}
), row=1, col=2)

# Terminal Percentage
terminal_pct = terminal / total_orders * 100
fig.add_trace(go.Indicator(
    mode="gauge+number",
    value=terminal_pct,
    gauge={
        'axis': {'range': [0, 100]},
        'bar': {'color': '#3498DB'},
        'steps': [
            {'range': [0, 33], 'color': '#E3F2FD'},
            {'range': [33, 66], 'color': '#BBDEFB'},
            {'range': [66, 100], 'color': '#90CAF9'}
        ]
    },
    number={'suffix': '%'}
), row=1, col=3)

fig.update_layout(height=350, title={'text': 'Key Performance Indicators', 'x': 0.5})
fig.show()

## 4. Pipeline Funnel Visualization

In [ ]:
# Create funnel stages
funnel_stages = [
    ('All Orders', total_orders),
    ('Acknowledged', total_orders - analysis_df[analysis_df[status_col] == 'Pending Acknowledgement'].shape[0]),
    ('Submitted', (analysis_df[status_col].isin(['Pending Approval', 'Approved', 'Rejected', 'Objected'])).sum()),
    ('Approved', approved)
]

fig = go.Figure(go.Funnel(
    y=[s[0] for s in funnel_stages],
    x=[s[1] for s in funnel_stages],
    textinfo="value+percent initial",
    marker={
        'color': ['#3498DB', '#2ECC71', '#E67E22', '#4ECDC4']
    },
    connector={"line": {"color": "royalblue", "width": 2}}
))

fig.update_layout(
    title={'text': 'Order Pipeline Funnel', 'x': 0.5, 'font': {'size': 20}},
    height=450
)

fig.show()

## 5. Status Breakdown by Product

In [ ]:
# Status by product
product_col = COLUMN_MAP['product']

product_status = analysis_df.groupby([product_col, status_col]).size().unstack(fill_value=0)

# Get top 15 products by total orders
top_products = analysis_df[product_col].value_counts().head(15).index
product_status_top = product_status.loc[product_status.index.isin(top_products)]

# Calculate totals for sorting
product_status_top['_total'] = product_status_top.sum(axis=1)
product_status_top = product_status_top.sort_values('_total', ascending=True)
product_status_top = product_status_top.drop('_total', axis=1)

# Create stacked bar chart
fig = go.Figure()

for status in STATUS_ORDER:
    if status in product_status_top.columns:
        fig.add_trace(go.Bar(
            y=product_status_top.index,
            x=product_status_top[status],
            name=status,
            orientation='h',
            marker_color=color_map.get(status, '#BDC3C7')
        ))

fig.update_layout(
    title={'text': 'Order Status by Product', 'x': 0.5, 'font': {'size': 20}},
    barmode='stack',
    xaxis_title='Number of Orders',
    yaxis_title='Product',
    height=600,
    margin={'l': 200},
    legend={'orientation': 'h', 'y': 1.1}
)

fig.show()

In [ ]:
# Approval rate by product
product_stats = analysis_df.groupby(product_col).agg(
    total=('is_approved', 'count'),
    approved=('is_approved', 'sum'),
    rejected=('is_rejected', 'sum'),
    cancelled=('is_cancelled', 'sum'),
    objected=('is_objected', 'sum'),
    total_qty=('quantity', 'sum'),
    completed_qty=('completed_qty', 'sum')
).reset_index()

product_stats['approval_rate'] = (product_stats['approved'] / (product_stats['total'] - product_stats['cancelled']) * 100).round(1)
product_stats['completion_rate'] = np.where(
    product_stats['total_qty'] > 0,
    (product_stats['completed_qty'] / product_stats['total_qty'] * 100).round(1),
    0
)
product_stats = product_stats.sort_values('total', ascending=False)

print("\n📊 STATUS METRICS BY PRODUCT (Top 15)")
print("="*100)
display_df = product_stats.head(15)[[
    product_col, 'total', 'approved', 'rejected', 'objected', 'approval_rate', 'completion_rate'
]].copy()
display_df.columns = ['Product', 'Total', 'Approved', 'Rejected', 'Objected', 'Approval %', 'Completion %']
print(display_df.to_string(index=False))

## 6. Quantity Completion Analysis

In [ ]:
# Completion rate by product (horizontal bar)
completion_data = product_stats[product_stats['total_qty'] > 0].head(15).sort_values('completion_rate')

fig = go.Figure()

# Completed portion
fig.add_trace(go.Bar(
    y=completion_data[product_col],
    x=completion_data['completed_qty'],
    name='Completed',
    orientation='h',
    marker_color='#4ECDC4',
    text=completion_data['completion_rate'].apply(lambda x: f'{x:.0f}%'),
    textposition='inside'
))

# Remaining portion
fig.add_trace(go.Bar(
    y=completion_data[product_col],
    x=completion_data['total_qty'] - completion_data['completed_qty'],
    name='Remaining',
    orientation='h',
    marker_color='#E0E0E0'
))

fig.update_layout(
    title={'text': 'Quantity Completion by Product', 'x': 0.5, 'font': {'size': 20}},
    barmode='stack',
    xaxis_title='Quantity',
    yaxis_title='Product',
    height=500,
    margin={'l': 200},
    legend={'orientation': 'h', 'y': 1.05}
)

fig.show()

## 7. Rejection & Objection Analysis

In [ ]:
# Rejection reasons breakdown
reason_col = COLUMN_MAP.get('rejection_reason')

if reason_col and reason_col in analysis_df.columns:
    # Filter to rejected/objected orders with reasons
    reason_df = analysis_df[
        analysis_df[status_col].isin(['Rejected', 'Objected']) &
        analysis_df[reason_col].notna() &
        (analysis_df[reason_col].str.strip() != '')
    ]
    
    if len(reason_df) > 0:
        reason_counts = reason_df[reason_col].value_counts().head(10)
        
        fig = go.Figure(data=[go.Bar(
            x=reason_counts.values,
            y=reason_counts.index,
            orientation='h',
            marker_color='#FF6B6B',
            text=reason_counts.values,
            textposition='outside'
        )])
        
        fig.update_layout(
            title={'text': 'Top Rejection/Objection Reasons', 'x': 0.5, 'font': {'size': 20}},
            xaxis_title='Count',
            yaxis_title='Reason',
            height=400,
            margin={'l': 300},
            yaxis={'categoryorder': 'total ascending'}
        )
        
        fig.show()
    else:
        print("No rejection reasons data available.")
else:
    print("Rejection reason column not found in data.")

In [ ]:
# Rejection rate by product
rejection_data = product_stats[product_stats['rejected'] > 0].copy()
rejection_data['rejection_rate'] = (rejection_data['rejected'] / (rejection_data['total'] - rejection_data['cancelled']) * 100).round(1)
rejection_data = rejection_data.nlargest(15, 'rejection_rate')

if len(rejection_data) > 0:
    fig = go.Figure(data=[go.Bar(
        y=rejection_data[product_col],
        x=rejection_data['rejection_rate'],
        orientation='h',
        marker_color='#FF6B6B',
        text=rejection_data['rejection_rate'].apply(lambda x: f'{x:.1f}%'),
        textposition='outside'
    )])
    
    fig.update_layout(
        title={'text': 'Rejection Rate by Product (Top 15)', 'x': 0.5, 'font': {'size': 20}},
        xaxis_title='Rejection Rate %',
        yaxis_title='Product',
        height=500,
        margin={'l': 200},
        yaxis={'categoryorder': 'total ascending'}
    )
    
    fig.show()
else:
    print("No rejected orders found.")

## 8. Waterfall: Order Flow

In [ ]:
# Waterfall chart showing order flow
pending_ack = (analysis_df[status_col] == 'Pending Acknowledgement').sum()
ack = (analysis_df[status_col] == 'Acknowledge').sum()
waiting = (analysis_df[status_col] == 'Waiting for order submission').sum()
pending_app = (analysis_df[status_col] == 'Pending Approval').sum()

fig = go.Figure(go.Waterfall(
    name="Order Flow",
    orientation="v",
    measure=["absolute", "relative", "relative", "relative", "relative", "relative", "relative", "total"],
    x=["Total Orders", "→ Pending Ack", "→ Acknowledged", "→ Waiting", "→ Pending Approval", "→ Approved", "→ Rejected/Objected", "Terminal"],
    y=[total_orders, -pending_ack, -ack, -waiting, -pending_app, -approved, -(rejected + objected), 0],
    connector={"line": {"color": "rgb(63, 63, 63)"}},
    text=[f"{total_orders}", f"-{pending_ack}", f"-{ack}", f"-{waiting}", f"-{pending_app}", f"-{approved}", f"-{rejected + objected}", f"{terminal}"],
    textposition="outside",
    decreasing={"marker": {"color": "#FF6B6B"}},
    increasing={"marker": {"color": "#4ECDC4"}},
    totals={"marker": {"color": "#3498DB"}}
))

fig.update_layout(
    title={'text': 'Order Status Waterfall', 'x': 0.5, 'font': {'size': 20}},
    height=450,
    showlegend=False
)

fig.show()

## 9. Status Summary Table

In [ ]:
# Complete status summary table
status_summary = analysis_df.groupby(status_col).agg(
    count=('is_approved', 'count'),
    total_qty=('quantity', 'sum'),
    completed_qty=('completed_qty', 'sum')
).reset_index()

status_summary['percentage'] = (status_summary['count'] / total_orders * 100).round(1)
status_summary['completion_rate'] = np.where(
    status_summary['total_qty'] > 0,
    (status_summary['completed_qty'] / status_summary['total_qty'] * 100).round(1),
    0
)

status_summary = status_summary.sort_values('count', ascending=False)
status_summary.columns = ['Status', 'Count', 'Total Qty', 'Completed Qty', '% of Total', 'Completion %']

print("\n📊 COMPLETE STATUS SUMMARY")
print("="*90)
print(status_summary.to_string(index=False))
print("="*90)
print(f"\nTOTAL: {total_orders:,} orders | {total_qty:,} quantity | {completed_qty:,} completed ({completion_rate:.1f}%)")

## 10. Export Results

In [ ]:
# Export to Excel
export_filename = f"order_status_metrics_{datetime.now().strftime('%Y%m%d_%H%M')}.xlsx"

with pd.ExcelWriter(export_filename, engine='openpyxl') as writer:
    # Summary metrics
    summary_metrics = pd.DataFrame({
        'Metric': ['Total Orders', 'Approved', 'Rejected', 'Cancelled', 'Objected', 'In Progress',
                   'Approval Rate', 'Rejection Rate', 'Objection Rate',
                   'Total Quantity', 'Completed Quantity', 'Completion Rate'],
        'Value': [total_orders, approved, rejected, cancelled, objected, in_progress,
                  f'{approval_rate:.1f}%', f'{rejection_rate:.1f}%', f'{objection_rate:.1f}%',
                  total_qty, completed_qty, f'{completion_rate:.1f}%']
    })
    summary_metrics.to_excel(writer, sheet_name='Summary', index=False)
    
    # Status breakdown
    status_summary.to_excel(writer, sheet_name='Status Summary', index=False)
    
    # Product breakdown
    product_stats.to_excel(writer, sheet_name='By Product', index=False)

print(f"\n✅ Results exported to: {export_filename}")
# files.download() - uncomment if using Colab
# files.download(export_filename)

---

## 📋 Summary

This notebook analyzed order status distribution and completion metrics:

| Metric | Description |
|--------|-------------|
| **Approval Rate** | % of non-cancelled orders that are approved |
| **Rejection Rate** | % of submitted orders that are rejected |
| **Objection Rate** | % of submitted orders with objections |
| **Completion Rate** | Completed qty / Total qty × 100 |
| **Terminal %** | % of orders in final status (Approved/Rejected/Cancelled) |

### Key Insights
1. Review products with low approval rates
2. Address common rejection reasons
3. Focus on products with high objection rates
4. Track completion metrics over time